# Тестовое задание

### Задача

Проезжающие машины в дождливое время образуют вокруг себя облако брызг, которые препятствуют нормальной езде беспилотного автомобиля.
Требуется предложить и реализовать модель, которая будет классифицировать точки на 3 класса:
- 0: фон
- 1: машина
- 2: шум (брызги и выхлопные газы)

Для обучения и тестирования модели рекомендуется использовать датасет [semantic_spray_dataset](https://github.innominds.com/aldipiroli/semantic_spray_dataset).  
В архиве приведен jupyter-ноутбук с примером загрузки облака с разметкой из этого датасета и их дальнейшей визуализацией.

Упрощения, которые можно использовать:
- допускается рассматривать точки только в непосредственной близости от автомобиля (например, x в пределах [-20; 20], y в пределах [-20, 20])
- допускается удалять точки земли по порогу, RANSAC'ом или иным способом
- допускается предварительно использовать алгоритм кластеризации (например, DBSCAN) и дальше определять классифицировать кластеры вместо точек.

Требования:
- Язык реализации python
- Время работы алгоритма: <= 50 мс (допускается до 100 мс)

Формат ответа:
- В качестве ответа требуется прикрепить ссылку на гитхаб с решением или загрузить код файлом или архивом.  
- Также нужно приложить 1-2 скрина с качеством работы модели.  
- Если модель обучалась, то будут полезны метрики обученной модели (mAP или иные)

Срок выполнения задачи - 1 неделя

In [1]:
import os
import time
import torch
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt

from abc import ABC, abstractmethod
from contextlib import contextmanager
from typing import List, Union, Tuple

from torch.utils.data import Dataset, DataLoader

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
@contextmanager
def timer():
    start = time.time()
    try:
        yield 
    finally:
        print(time.time() - start)

visualization_params = dict(
    front=[-0.23497, -0.6329, 0.7376],
    lookat=[0.0016, 0.0025, -1.1011],
    up=[0.5962, 0.5054, 0.6236],
    zoom=0.28
    )

def show(points, labels, bb_vehicle=False):
    """Функция для визулизации облака точек (3 класса точек)
    с возможностью визуализации бокса для клсса автомобиля"""
    colors = np.zeros((labels.size, 3))
    colors[labels == 0, 0] = 1. # background - red
    colors[labels == 1, 1] = 1. # foreground (vehicle) - green
    colors[labels == 2, 2] = 1. # noise - blue

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    pcd.colors = o3d.utility.Vector3dVector(colors)

    visuals = [pcd]
    if bb_vehicle:
        cluster = pcd.select_by_index(np.where(labels == 1)[0])
        bounding_box = cluster.get_axis_aligned_bounding_box()
        bounding_box.color = [0, 0, 0]
        visuals.append(bounding_box)

    o3d.visualization.draw_geometries(visuals, **visualization_params)  

Классы трансформации (преобразования) облака точек, которые будут использоваться в дальнейшем:

In [3]:
class PointCloudTransform(ABC):
    """Базовый класс преобразования облака точек"""
    def __init__(self):
        pass 

    def __call__(self, points, labels):
        return self.transform(points, labels)
    
    @abstractmethod
    def transform(self, points, labels):
        pass 

class RandomRotateZ(PointCloudTransform):
    """Класс поворота облака точек вокруг оси Oz на случайный угол"""
    def __init__(self):
        super(RandomRotateZ, self).__init__()

    def transform(self, points, labels):
        rotation_angle = np.random.uniform() * 2 * np.pi
        cos_val = np.cos(rotation_angle)
        sin_val = np.sin(rotation_angle)
        rotation_matrix = np.array([[cos_val, sin_val, 0],
                                    [-sin_val, cos_val, 0],
                                    [0, 0, 1]])
        points[:, :3] = np.dot(points[:, :3], rotation_matrix)
        return points, labels 

class CenterCropXY(PointCloudTransform):
    """Класс вырезания части облака точек заданного размера (по осям Ox и Oy)"""
    def __init__(self, 
                 sizes: Union[List[float], Tuple[float], np.ndarray]):
        super(CenterCropXY, self).__init__()
        self.sizes = sizes

    def transform(self, points, labels):
        mask = (np.abs(points[:, 0]) <= self.sizes[0]) & (np.abs(points[:, 1]) <= self.sizes[1])
        return points[mask, :], labels[mask]

class CenterCrop(CenterCropXY):
    """Класс вырезания симметричной части облака точек заданного размера (по осям Ox и Oy)"""
    def __init__(self, size):
        super(CenterCrop, self).__init__([size, size])
    
class MinMaxScaler(PointCloudTransform):
    """Класс Min-Max нормализации облака точек"""
    def __init__(self, 
                 mins: Union[List[float], Tuple[float], np.ndarray], 
                 maxs: Union[List[float], Tuple[float], np.ndarray]):
        self.mins = np.array(mins)
        self.maxs = np.array(maxs)
    
    def transform(self, points, labels):
        # points[:, :3] = (points[:, :3] - self.mins) / (self.maxs - self.mins)
        n = points.shape[1]
        points = (points - self.mins[:n]) / (self.maxs[:n] - self.mins[:n])
        return points, labels

class Compose(PointCloudTransform):
    """Класс последовательного применения трансформаций облака точек"""
    def __init__(self, transfroms: List[PointCloudTransform]):
        self.transfroms = transfroms

    def transform(self, points, labels):
        for transfrom in self.transfroms:
            points, labels = transfrom(points, labels)
        return points, labels

Класс, определяющий датасет SemanticSprayDataset

In [4]:
class SemanticSprayDataset(Dataset):
    """Датасет, определяющий облако точек для обучениния молелей
    Разбиение на обучающую и валидационную выборки определяется файлами ImageSets/train.txt и ImageSets/test.txt,
    представленными в репозитори датасета https://github.innominds.com/aldipiroli/semantic_spray_dataset
    Часть определения датасета позаимствована из вышеуказанного репозитория. 
    Params:
        root_path (str) - путь к данным 
        split (str) - train или test
        return_xyz (bool) - флаг, определяющий вернуть все каналы или только координатные 
        sample_size (int) - размер возвращаемого облака точек; точки случайно сэмплируются 
            из исходного облака с возвращением или без в зависимости от размера исходного облака.
            Если значение параметра None, то будет возвращено всё облако точек
        labelweights - веса лейблов
        transfroms - преобразование или набор преобразований облака точек
    """
    def __init__(self, 
                 root_path: str, 
                 split: str = "train",
                 return_xyz: bool = True,
                 sample_size: int = 4096,
                 labelweights: Union[np.ndarray, List[float]] = None,
                 transfroms: Union[Compose, PointCloudTransform] = None):
        self.root_path = root_path
        self.split = split
        self.return_xyz = return_xyz

        split_dir = os.path.join(self.root_path, "ImageSets", self.split + ".txt")
        scenes_list = [x.strip() for x in open(split_dir).readlines()]

        self.sample_size = sample_size
        self.transfroms = transfroms
        if labelweights:
            self.labelweights = 1 - np.array(labelweights)
        self.sample_id_list = self.get_data_samples(scenes_list)

    def get_data_samples(self, scenes_list):
        all_samples = []
        for sample_id in scenes_list:
            velo_files = sorted(next(os.walk(os.path.join(self.root_path, sample_id, "velodyne")), (None, None, []))[2])
            for curr_id in velo_files:
                scan_id = curr_id.split(".")[0]
                all_samples.append(os.path.join(self.root_path, sample_id, scan_id))
        return all_samples

    def load_data(self, scene_path, scan_id):
        velo_path = os.path.join(scene_path, "velodyne", scan_id + ".bin")
        points = np.fromfile(velo_path, np.float32).reshape(-1, 5)
        labels_path = os.path.join(scene_path, "labels", scan_id + ".label")
        labels = np.fromfile(labels_path, np.int32).reshape(-1)  
        if self.return_xyz:
            return points[:, :3], labels 
        else:
            return points, labels 

    def __len__(self):
        return len(self.sample_id_list)

    def __getitem__(self, index):
        data_path = os.path.join(self.sample_id_list[index])
        scan_id = data_path.split("\\")[-1] 
        scene_path = data_path[:-6]
        points, labels = self.load_data(scene_path, scan_id)
        if self.transfroms:
            points, labels = self.transfroms(points, labels)

        if self.sample_size is None:
            return points, labels
        
        if labels.size >= self.sample_size:
            # Сэмлирование с определенными вероятностями для разных классов [не используется в финальном варианте]
            # `Если класс малочисленный, то вероятность его семлирования выше`
            # sampling_weights = np.array([self.labelweights[i] for i in labels]) 
            # sampling_weights = sampling_weights / np.sum(sampling_weights)
            # points_idxs = np.random.choice(labels.size, self.sample_size, replace=False, p=sampling_weights)
            points_idxs = np.random.choice(labels.size, self.sample_size, replace=False)
        else:
            points_idxs = np.random.choice(labels.size, self.sample_size, replace=True)

        points, labels = points[points_idxs], labels[points_idxs]

        return points, labels

### Первый метод (RANSAC planar segmentation + DBSCAN clustering)

`Сразу стоит отметить, что данный метод предполагает достаточно много условностей и не был проанализирован до конца. Предпочтение было отдано другому методу, рассмотренному в дальнейшем.`

Предобработка:
- из исходного облака точек вырезается область, для которой $|x| < 30$ и $|y| < 30$ (если вырезать область $|x| < 20$ и $|y| < 20$, то в неё почти никогда не попадает транспортное средство)

Предположения (они же упрощения):
- предполагается, что машина и шумы находятся в узкой полосе  $-10 < y < 5$ (слева болшее расстояние, так как движение правосторенне). Также определять данную область можно с помощью кластеризации DBSCAN, однако данный метод на практике работает дольше 50ms и сильно чувствителен к гиперпараметрам.
- предполагается, что машина - достаточно плотный кластер, который можно отделить от шумов с помощью DBSCAN с достаточно малым парметром $\varepsilon$ (как будет видно далее, предположение не всегда корректно; например, при очень большом количестве плотных шумов)
- предполагается, что центральные точки $|x| < 2$ и $|y| < 2$, отвечающиее за транспортное средство с лидаром, можно по-умолчанию относить к классу 0. В далее представленных методах данный случай не будет обрабатываться отдельно.  

Далее будут предложены 2 достаточно близких по сути варианта, отличающихся порядком выполнения действий. 

#### Вариант 1:
1) С использованием алгоритма RANSAC из облака точек отделяются точки поверхности 
2) Отделяется центральная полоса (дорога), для которой $-10 < y < 5$. Точки вне дороги относятся к классу 0 (фон)
3) Точки внутри полосы (дороги) кластеризуются с использованием алгоритма DBSCAN с малым значением параметра  $\varepsilon$ для отделения машины от шумов 

In [68]:
def segment_cloud(points, labels, transfrom=None):
    # ================================================
    # crop central area 30 x 30 and create point cloud 
    # ================================================
    if transfrom:
        points, labels = transfrom(points, labels)
    points, labels = transfrom(points, labels)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])

    # ================================================
    # remove surface area with RANSAC
    # ================================================
    dist = np.mean(pcd.compute_nearest_neighbor_distance())
    distance_threshold = 2 * dist 
    _, surface_idxs = pcd.segment_plane(distance_threshold=distance_threshold,
                                        ransac_n=10,
                                        num_iterations=1000)

    surface_cloud = pcd.select_by_index(surface_idxs)
    surface_cloud.paint_uniform_color([1, 0, 0]) # surface - red 
    non_surface_cloud = pcd.select_by_index(surface_idxs, invert=True)
    non_surface_cloud.paint_uniform_color([0, 0, 0])

    # ================================================
    # crop narrow central area by -7 < |y| < 5 
    # Due to right-hand traffic left crop is greater
    # ================================================
    non_surface_points = np.asarray(non_surface_cloud.points)

    crop_y_left, crop_y_roght = -7, 5
    mask = np.where(((non_surface_points[:, 1] > crop_y_left) & 
                    (non_surface_points[:, 1] < crop_y_roght)))[0]

    inner_cloud = non_surface_cloud.select_by_index(mask)
    inner_cloud.paint_uniform_color([0, 0, 1]) 
    outer_cloud = non_surface_cloud.select_by_index(mask, invert=True)
    outer_cloud.paint_uniform_color([1, 0, 0]) 

    # ================================================
    # DBSCAN clustering for inner_cloud (narrow road area)
    # ================================================
    dist = np.mean(inner_cloud.compute_nearest_neighbor_distance())
    eps, min_points = 4 * dist, 30
    dbscan_labels = np.array(inner_cloud.cluster_dbscan(eps=eps, 
                                                        min_points=min_points))
    dbscan_colors = np.zeros((dbscan_labels.shape[0], 3))
    dbscan_colors[dbscan_labels < 0 , 2] = 1 # noise - blue
    dbscan_colors[dbscan_labels >= 0, 1] = 1 # foreground (vehicle) - green
    inner_cloud.colors = o3d.utility.Vector3dVector(dbscan_colors[:, :3])

    # ================================================
    # Bounding Boxes for inner_cloud
    # ================================================
    bounding_boxes = []
    for cluster_idx in np.unique(dbscan_labels):
        if cluster_idx == -1:
            continue
        cluster = inner_cloud.select_by_index(np.where(dbscan_labels == cluster_idx)[0])
        bounding_box = cluster.get_axis_aligned_bounding_box()
        bounding_box.color = [0, 0, 0]
        bounding_boxes.append(bounding_box)

    # ================================================
    # Visualization
    # ================================================
    visuals = [surface_cloud, inner_cloud, outer_cloud, *bounding_boxes]
    return visuals

In [57]:
cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)

transfrom = CenterCrop(30)
points, labels = transfrom(points, labels)
show(points, labels, bb_vehicle=True)

o3d.visualization.draw_geometries(segment_cloud(points, labels, transfrom=CenterCrop(30)), **visualization_params)

In [59]:
train_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "train",
    return_xyz=True,
    sample_size=None,
    transfroms=CenterCrop(30)
)

test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=True,
    sample_size=None,
    transfroms=CenterCrop(30)
)

show(*train_dataset[2399], bb_vehicle=True)
o3d.visualization.draw_geometries(segment_cloud(*train_dataset[2399], transfrom=CenterCrop(30)), **visualization_params)

show(*test_dataset[1399], bb_vehicle=True)
o3d.visualization.draw_geometries(segment_cloud(*test_dataset[1399], transfrom=CenterCrop(30)), **visualization_params)

|Оригинальное облако|RANSAC + DBSCAN, V1 (предсказание)|
|-|-|
|Тестовый сэмпл|Тестовый сэмпл (предсказание)|
|![alt](images/ransac_v1_orig.png) | ![alt](images/ransac_v1_pred.png)|
|Сэмпл из тестового датсета|Сэмпл из тестового датсета (предсказание)|
|![alt](images/ransac_v1_orig_test.png) | ![alt](images/ransac_v1_pred_test.png)|
|Сэмпл из обучающего датсета|Сэмпл из обучающего датсета (предсказание)|
|![alt](images/ransac_v1_orig_train.png) | ![alt](images/ransac_v1_pred_train.png)|

In [69]:
%timeit -n 10 -r 10 segment_cloud(points, labels, transfrom=CenterCrop(20))
%timeit -n 10 -r 10 segment_cloud(points, labels, transfrom=CenterCrop(30))

27.7 ms ± 951 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
34 ms ± 1.4 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


Среднее время на одно предсказание для модели RANSAC + DBSCAN (Intel i9 12900K):
1) для облака точек в области $|x| < 20$ и $|y| < 20$ ~ 28ms
2) для облака точек в области $|x| < 30$ и $|y| < 30$ ~ 34ms 

#### Вариант 2:
1) Отделяется центральная полоса (дорога), для которой $-10 < y < 5$. Точки вне дороги относятся к классу 0 (фон)
2) С использованием алгоритма RANSAC из центраьной полосы отделяются точки поверхности 
3) Точки внутри полосы (дороги) и над поверхностью кластеризуются с использованием алгоритма DBSCAN с малым значением параметра $\varepsilon$ для отделения машины от шумов 

In [70]:
def segment_cloud(points, labels, transfrom=None):
    # ================================================
    # crop central area 30 x 30 and create point cloud 
    # ================================================
    if transfrom:
        points, labels = transfrom(points, labels)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])

    # ================================================
    # crop narrow central area by -7 < |y| < 5 
    # Due to right-hand traffic left crop is greater
    # ================================================
    crop_y_left, crop_y_roght = -7, 5
    mask = np.where(((points[:, 1] > crop_y_left) & 
                     (points[:, 1] < crop_y_roght)))[0]

    inner_cloud = pcd.select_by_index(mask)
    inner_cloud.paint_uniform_color([0, 0, 1]) # inner - blue
    outer_cloud = pcd.select_by_index(mask, invert=True)
    outer_cloud.paint_uniform_color([1, 0, 0]) # outer - blue

    # ================================================
    # remove surface area for inner_cloud with RANSAC 
    # ================================================
    dist = np.mean(inner_cloud.compute_nearest_neighbor_distance())
    distance_threshold = 2 * dist 
    _, surface_idxs = inner_cloud.segment_plane(distance_threshold=distance_threshold,
                                                ransac_n=10,
                                                num_iterations=1000)

    surface_cloud = inner_cloud.select_by_index(surface_idxs)
    surface_cloud.paint_uniform_color([1, 0, 0]) # surface - red 
    non_surface_cloud = inner_cloud.select_by_index(surface_idxs, invert=True)
    non_surface_cloud.paint_uniform_color([0, 0, 0]) # non surface - red 

    # ================================================
    # DBSCAN clustering for non_surface_cloud
    # ================================================
    dist = np.mean(non_surface_cloud.compute_nearest_neighbor_distance())
    eps, min_points = 4 * dist, 30
    dbscan_labels = np.array(non_surface_cloud.cluster_dbscan(eps=eps, 
                                                              min_points=min_points))
    dbscan_colors = np.zeros((dbscan_labels.shape[0], 3))
    dbscan_colors[dbscan_labels < 0 , 2] = 1 # noise - blue
    dbscan_colors[dbscan_labels >= 0, 1] = 1 # foreground (vehicle) - green
    non_surface_cloud.colors = o3d.utility.Vector3dVector(dbscan_colors[:, :3])

    # ================================================
    # Bounding Boxes for non_surface_cloud
    # ================================================
    bounding_boxes = []
    for cluster_idx in np.unique(dbscan_labels):
        if cluster_idx == -1:
            continue
        cluster = non_surface_cloud.select_by_index(np.where(dbscan_labels == cluster_idx)[0])
        bounding_box = cluster.get_axis_aligned_bounding_box()
        bounding_box.color = [0, 0, 0]
        bounding_boxes.append(bounding_box)

    # ================================================
    # Visualization
    # ================================================       
    visuals = [outer_cloud, surface_cloud, non_surface_cloud, *bounding_boxes]
    return visuals

In [62]:
cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)

transfrom = CenterCrop(30)
points, labels = transfrom(points, labels)
show(points, labels, bb_vehicle=True)

o3d.visualization.draw_geometries(segment_cloud(points, labels, transfrom=CenterCrop(30)), **visualization_params)

In [63]:
train_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "train",
    return_xyz=True,
    sample_size=None,
    transfroms=CenterCrop(30)
)

test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=True,
    sample_size=None,
    transfroms=CenterCrop(30)
)

show(*train_dataset[2399], bb_vehicle=True)
o3d.visualization.draw_geometries(segment_cloud(*train_dataset[2399], transfrom=CenterCrop(30)), **visualization_params)

show(*test_dataset[1399], bb_vehicle=True)
o3d.visualization.draw_geometries(segment_cloud(*test_dataset[1399], transfrom=CenterCrop(30)), **visualization_params)

|Оригинальное облако|RANSAC + DBSCAN, V2 (предсказание)|
|-|-|
|Тестовый сэмпл|Тестовый сэмпл (предсказание)|
|![alt](images/ransac_v2_orig.png) | ![alt](images/ransac_v2_pred.png)|
|Сэмпл из тестового датсета|Сэмпл из тестового датсета (предсказание)|
|![alt](images/ransac_v2_orig_test.png) | ![alt](images/ransac_v2_pred_test.png)|
|Сэмпл из обучающего датсета|Сэмпл из обучающего датсета (предсказание)|
|![alt](images/ransac_v2_orig_train.png) | ![alt](images/ransac_v2_pred_train.png)|

In [71]:
%timeit -n 10 -r 10 segment_cloud(points, labels, transfrom=CenterCrop(20))
%timeit -n 10 -r 10 segment_cloud(points, labels, transfrom=CenterCrop(30))

25.8 ms ± 1.13 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
31.6 ms ± 1.21 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


Среднее время на одно предсказание для модели RANSAC + DBSCAN (Intel i9 12900K):
1) для облака точек в области $|x| < 20$ и $|y| < 20$ ~ 26ms
2) для облака точек в области $|x| < 30$ и $|y| < 30$ ~ 32ms 

## PointNet++ и DGCNN

Далее будут рассмотрены 2 deep learning подхода к сегментации облака точек (будут рассмотрены готовые реализации моделей на pytorch с некоторыми доработками). 

Код модели PointNet++ и цикла её обучения были взяты из репозитория [PointNet++](https://github.com/yanx27/Pointnet_Pointnet2_pytorch)

Код модели DGCNN и цикла её обучения были взяты из репозитория [DGCNN](https://github.com/antao97/dgcnn.pytorch)

Разбиение датасета на обучающую и тестовую части было взято из репозитория [semantic_spray_dataset](https://github.innominds.com/aldipiroli/semantic_spray_dataset), полный датасет был загружен из источника [Data](https://oparu.uni-ulm.de/xmlui/handle/123456789/48891)

Помимо координатных каналов xyz в моделях будут использоваться каналы ring и intensity -> всего 5 каналов

Определение некоторых характеристик датасета:

1) минимальные и максимальные значения координат для нормировки ($|x| < 30$ и $|y| < 30$ вследстивие преобразования `CenterCrop(30)`)
2) количество точек для различных лейблов (для получения веса каждго класса с целью дальнейшего использования в функции потерь)

In [5]:
import tqdm 
    
dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "train",
    return_xyz=False,
    sample_size=4096,
    transfroms=CenterCrop(30)
)

max_all, min_all = [], []
labelweights = np.zeros(3)

for (points, labels) in tqdm.tqdm(dataset, total=len(dataset)):
    tmp, _ = np.histogram(labels, range(4))
    labelweights += tmp   
    max_all.append(np.amax(points, axis=0)), min_all.append(np.amin(points, axis=0))

max_all_np = np.vstack(max_all)
min_all_np = np.vstack(min_all)

maxs = np.amax(max_all_np, axis=0)
mins = np.amin(min_all_np, axis=0)
labelweights = labelweights / np.sum(labelweights)

100%|██████████| 8389/8389 [00:09<00:00, 929.61it/s]


## Второй метод (модель PointNet++)

In [6]:
from pointnet_pp.train import train


class Args:
    BATCH_SIZE = 16
    NUM_CLASSES = 3
    LEARNING_RATE_CLIP = 1e-5
    MOMENTUM_ORIGINAL = 0.1
    MOMENTUM_DECCAY = 0.5
    STEP_SIZE = 5
    MOMENTUM_DECCAY_STEP = STEP_SIZE
    N_EPOCHS = 17
    LR_DECAY = 0.7
    LEARNING_RATE = 1e-3
    NUM_POINT = 4096
    IN_CHANNELS = 5
    from_checkpoint = False
    maxs = maxs
    mins = mins
    labelweights = labelweights

transfroms = Compose([   
    # RandomRotateZ(),
    CenterCrop(30),
    MinMaxScaler(mins=Args.mins, maxs=Args.maxs) 
])
    
train_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "train",
    return_xyz=False,
    sample_size=Args.NUM_POINT,
    transfroms=transfroms
)

test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=False,
    sample_size=Args.NUM_POINT,
    transfroms=transfroms
)

train_loader = DataLoader(train_dataset, batch_size=Args.BATCH_SIZE, shuffle=True, 
                          drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=Args.BATCH_SIZE, shuffle=False, 
                         drop_last=True, pin_memory=True)

In [60]:
train(train_loader, test_loader, Args)

100%|██████████| 524/524 [03:58<00:00,  2.20it/s]


Train 0, loss: 0.055339, train acc: 0.990479


100%|██████████| 541/541 [04:23<00:00,  2.05it/s]


Test 0, loss: 0.017476, test acc: 0.996953, test avg acc: 0.983313, test iou: 0.933597
------- IoU --------
class background     weight: 0.971, IoU: 0.997 
class foreground     weight: 0.011, IoU: 0.927 
class noise          weight: 0.018, IoU: 0.877 



100%|██████████| 524/524 [03:57<00:00,  2.21it/s]


...

Test 15, loss: 0.010420, test acc: 0.998056, test avg acc: 0.991137, test iou: 0.960153


100%|██████████| 524/524 [03:59<00:00,  2.18it/s]


Train 16, loss: 0.010530, train acc: 0.997899


100%|██████████| 541/541 [03:45<00:00,  2.39it/s]

Test 16, loss: 0.010448, test acc: 0.998155, test avg acc: 0.990928, test iou: 0.961291


In [29]:
from pointnet_pp.model import get_model


model = get_model(Args.IN_CHANNELS, Args.NUM_CLASSES).cuda()
model.load_state_dict(torch.load("./models/pointnet_pp_model_best.pth")["model_state_dict"])
model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

966851


In [11]:
from pointnet_pp.model import get_model


model = get_model(Args.IN_CHANNELS, Args.NUM_CLASSES).cuda()
model.load_state_dict(torch.load("./models/pointnet_pp_model_best.pth")["model_state_dict"])
model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

@torch.no_grad()
def predict_one(model, points, labels, args=Args, transfrom=CenterCrop(30)):
    """Предсказание производится батчами размером по args.NUM_POINT для PointNet++. 
    Данный размер облака точек использовался во время обучения. 
    """
    points = points[:, :args.IN_CHANNELS]
    if transfrom:
        points, labels = transfrom(points, labels)

    idxs = np.random.permutation(np.arange(1, len(labels)))
    points, labels = points[idxs, :], labels[idxs]

    n_points = points.shape[0]
    prop_size = args.NUM_POINT - (n_points % args.NUM_POINT)
    points_prop = np.zeros((n_points + prop_size, points.shape[1]))
    points_prop[:n_points, :] = points
    points_prop[n_points:, :] = points[np.random.choice(labels.size, prop_size, replace=False), :]
    
    preds_all = []
    for i in range(0, n_points, args.NUM_POINT):

        points_sample = points_prop[i:i+args.NUM_POINT, :]
        points_sample = MinMaxScaler(mins=args.mins, maxs=args.maxs)(points_sample, labels)[0][None, :, :]

        points_sample = torch.Tensor(points_sample)
        points_sample = points_sample.float().cuda()
        points_sample = points_sample.transpose(2, 1)

        seg_pred, _ = model(points_sample)
        pred_val = seg_pred.contiguous().cpu().data.numpy()

        pred_val = np.argmax(pred_val, 2)[0]
        preds_all.append(pred_val)

    return points, np.hstack(preds_all)[:n_points]

966851


In [12]:
cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)

points_s, pred_labels = predict_one(model, points, labels, transfrom=CenterCrop(30))
    
transfrom = CenterCrop(30)
points, labels = transfrom(points, labels)
show(points, labels, bb_vehicle=True)

show(points_s, pred_labels, bb_vehicle=True)

In [48]:
test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=False,
    sample_size=None,
    transfroms=CenterCrop(30)
)

points, labels = test_dataset[1399]

points_s, pred_labels = predict_one(model, points, labels, transfrom=CenterCrop(30))
    
show(points, labels, bb_vehicle=True)
show(points_s, pred_labels, bb_vehicle=True)

|Оригинальное облако|PointNet++ (предсказание)|
|-|-|
|Тестовый сэмпл|Тестовый сэмпл (предсказание)|
|![alt](images/pointnet_orig.png) | ![alt](images/pointnet_pred.png)|
|Сэмпл из тестового датсета|Сэмпл из тестового датсета (предсказание)|
|![alt](images/pointnet_orig_ds.png) | ![alt](images/pointnet_pred_ds.png)|

In [13]:
%timeit -n 1 -r 10 predict_one(model, points, labels, transfrom=CenterCrop(20))
%timeit -n 1 -r 10 predict_one(model, points, labels, transfrom=CenterCrop(30))

1.35 s ± 74.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.62 s ± 18.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


Среднее время на одно предсказание для модели PointNet++ (RTX 2070 SUPER):
1) для облака точек в области $|x| < 20$ и $|y| < 20$ ~ 1.3s
1) для облака точек в области $|x| < 30$ и $|y| < 30$ ~ 1.6s

### Вывод по PointNet++

Плюсы: 
- неплохая точность алгоритма

Минусы:
- очень долгий инференс > 1s

Метрики качества:

|Класс|Вес класса|IoU|
|:-|:-|:-|
|background|0.971|0.998|
|foreground|0.011|0.965|
|noise|0.018|0.925|

## Третий метод (модель DGCNN)

In [7]:
from dgcnn.train import train


class Args:
    lr = 1e-3
    epochs = 17
    from_checkpoint = False
    scheduler = 'step'
    step_size = 5
    emb_dims = 1024
    k = 20
    in_channels = 5
    num_points = 2048
    batch_size = 16
    test_batch_size = 16
    dropout = 0.5
    num_classes = 3
    maxs = maxs
    mins = mins
    labelweights = labelweights

transfroms = Compose([   
    # RandomRotateZ(),
    CenterCrop(30),
    MinMaxScaler(mins=Args.mins, maxs=Args.maxs) 
])
    
train_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "train",
    return_xyz=False,
    sample_size=Args.num_points,
    transfroms=transfroms
)

test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=False,
    sample_size=Args.num_points,
    transfroms=transfroms
)

train_loader = DataLoader(train_dataset, batch_size=Args.batch_size, shuffle=True, 
                          drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=Args.test_batch_size, shuffle=False, 
                         drop_last=True, pin_memory=True)

In [8]:
train(train_loader, test_loader, Args)

100%|██████████| 524/524 [01:53<00:00,  4.64it/s]


Train 0, loss: 0.047669, train acc: 0.990073, train avg acc: 0.966504, train iou: 0.824045


100%|██████████| 541/541 [01:05<00:00,  8.22it/s]


Test 0, loss: 0.017520, test acc: 0.997761, test avg acc: 0.975307, test iou: 0.950558
------- IoU --------
class background     weight: 0.971, IoU: 0.998 
class foreground     weight: 0.011, IoU: 0.952 
class noise          weight: 0.018, IoU: 0.902 



100%|██████████| 524/524 [01:48<00:00,  4.85it/s]


...

Test 15, loss: 0.009327, test acc: 0.998472, test avg acc: 0.990673, test iou: 0.967194


100%|██████████| 524/524 [01:52<00:00,  4.67it/s]


Train 16, loss: 0.007121, train acc: 0.998572, train avg acc: 0.994793, train iou: 0.972885


100%|██████████| 541/541 [01:05<00:00,  8.26it/s]


Test 16, loss: 0.009272, test acc: 0.998451, test avg acc: 0.991340, test iou: 0.967424


In [9]:
from dgcnn.model import DGCNN_semseg


model = DGCNN_semseg(Args).cuda()
model.load_state_dict(torch.load("./models/dgcnn_model_best.t7"))
model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

@torch.no_grad()
def predict_one(model, points, labels, args=Args, transfrom=CenterCrop(30)):
    """Предсказание производится батчами размером по args.NUM_POINT для DGCNN. 
    Данный размер облака точек использовался во время обучения. 
    """
    points = points[:, :args.in_channels]
    if transfrom:
        points, labels = transfrom(points, labels)

    idxs = np.random.permutation(np.arange(1, len(labels)))
    points, labels = points[idxs, :], labels[idxs]

    n_points = points.shape[0]
    prop_size = args.num_points - (n_points % args.num_points)
    points_prop = np.zeros((n_points + prop_size, points.shape[1]))
    points_prop[:n_points, :] = points
    points_prop[n_points:, :] = points[np.random.choice(labels.size, prop_size, replace=False), :]
    
    preds_all = []
    for i in range(0, n_points, args.num_points):

        points_sample = points[i:i+args.num_points, :]
        points_sample = MinMaxScaler(mins=args.mins, maxs=args.maxs)(points_sample, labels)[0][None, :, :]

        points_sample = torch.Tensor(points_sample)
        points_sample = points_sample.float().cuda()
        points_sample = points_sample.permute(0, 2, 1)

        seg_pred = model(points_sample)
        seg_pred = seg_pred.permute(0, 2, 1).contiguous()
        pred_val = seg_pred.view(-1, Args.num_classes).detach().cpu().numpy()
        pred_val = np.argmax(pred_val, 1)

        preds_all.append(pred_val)

    return points, np.hstack(preds_all)[:n_points]

980480


In [41]:
cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)

points_s, pred_labels = predict_one(model, points, labels, transfrom=CenterCrop(30))
    
transfrom = CenterCrop(30)
points, labels = transfrom(points, labels)
show(points, labels, bb_vehicle=True)

show(points_s, pred_labels, bb_vehicle=True)

In [42]:
test_dataset = SemanticSprayDataset(
    "./SemanticSprayDataset/",
    "test",
    return_xyz=False,
    sample_size=None,
    transfroms=CenterCrop(30)
)

points, labels = test_dataset[1399]

points_s, pred_labels = predict_one(model, points, labels, transfrom=CenterCrop(30))
    
show(points, labels, bb_vehicle=True)
show(points_s, pred_labels, bb_vehicle=True)

|Оригинальное облако|DGCNN (предсказание)|
|-|-|
|Тестовый сэмпл|Тестовый сэмпл (предсказание)|
|![alt](images/dgcnn_orig.png) | ![alt](images/dgcnn_pred.png)|
|Сэмпл из тестового датсета|Сэмпл из тестового датсета (предсказание)|
|![alt](images/dgcnn_orig_ds.png) | ![alt](images/dgcnn_pred_ds.png)|

In [44]:
%timeit -n 5 -r 100 predict_one(model, points, labels, transfrom=CenterCrop(20))
%timeit -n 5 -r 100 predict_one(model, points, labels, transfrom=CenterCrop(30))

49.5 ms ± 1.19 ms per loop (mean ± std. dev. of 100 runs, 5 loops each)
68.9 ms ± 1.46 ms per loop (mean ± std. dev. of 100 runs, 5 loops each)


Среднее время на одно предсказание для модели DGCNN (RTX 2070 SUPER):
1) для облака точек в области $|x| < 20$ и $|y| < 20$ ~ 49ms
2) для облака точек в области $|x| < 30$ и $|y| < 30$ ~ 68ms 

### Вывод по DGCNN

Плюсы: 
1) неплохая точность алгоритма 
2) приемлемый инференс `~49ms` на ограниченном облаке точек

Минусы:
1) всё ещё медленный инференс для полного облака точек

Метрики качества:

|Класс|Вес класса|IoU|
|:-|:-|:-|
|background|0.971|0.999|
|foreground|0.011|0.972|
|noise|0.018|0.933|

## Вывод:

В качестве финального алгоритма предлагается использование модели DGCNN, которая в рамках данной задачи удовлетворяет поставленным условиям. 
Модель показывает высокое качество сегментации для всех трёх классов (по метрике IoU и визуально), а также инференс модели на ограниченном облаке точек менее 50ms. 

In [72]:
!nvidia-smi

Thu Jan  4 19:52:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| 53%   32C    P8               8W / 215W |   6744MiB /  8192MiB |     18%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--